In [1]:
print('hello')


hello


In [ ]:
import os
os.system('echo hello')
os.system('squeue --me')

hello
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           9410600 interacti sys/dash danieljo  R      44:06      1 nid002279


0

In [3]:
import dask
print(dask.__version__)

2025.1.0


In [11]:
import os
os.system('export DASK_DISTRIBUTED__NANNY__STARTUP_TIMEOUT=60')

0

In [22]:


prologue_commands = [
    "echo 'Loading cray-python'"
    "module load cray-python"
    "echo 'Loading Python Enviroment'",
    "source /scratch/project_462000451/run/venv_lumi_cray_python2/bin/activate",
]
cluster = SLURMCluster(
    n_workers=2,
    queue='small',
    account="project_462000451",
    cores=1,
    job_cpu=2,
    memory="1GB",
    interface= "nmn0",
    job_script_prologue=prologue_commands
)
#['lo', 'nmn0', 'hsn1', 'hsn0', 'can0', 'bond0', 'net1', 'net2', 'net3']
# cluster.scale(jobs=2)  # ask for 10 jobs
# cluster.scale()

/project/project_462000451/enchanted_container_lumi3/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39333 instead
  warnings.warn(


In [2]:
from dask_jobqueue import SLURMCluster

# custom_worker_command = "bash -c 'export PATH=/project/project_462000451/enchanted_container_lumi3/bin:$PATH && python -m distributed.cli.dask_worker'"

# prologue_commands = [
#     # "echo 'Loading cray-python'"
#     # "module load cray-python"
#     # "echo 'Loading Python Enviroment'",
#     # "source /scratch/project_462000451/run/venv_lumi_cray_python2/bin/activate",

#     # "echo 'checking squeue --me'",
#     # 'squeue --me',
#     # "echo 'checking dask can be imported'",
#     # "python3 /project/project_462000451/DEEPlasma/check_dask_imported.py",
#     # "python3 -m distributed.cli.dask_worker"
#     # "echo 'exporting path for container'",
#     # "export PATH='/project/project_462000451/enchanted_container_lumi3/bin:$PATH'"
# ]

job_script_prologue = [
    # ''
    'export PATH="/project/project_462000451/enchanted_container_lumi3/bin:$PATH"',
    'cd /project/project_462000451/enchanted-surrogates/',
    'export PYTHONPATH=$PYTHONPATH:/project/project_462000451/enchanted-surrogates/src' # NB: to use the enchanted-surrogate library
]

cluster = SLURMCluster(
    # n_workers = 2,
    queue='small',
    account="project_462000451",
    cores=10, #Total number of cores spread evenly among the workers
    processes=1,
    memory="1GB", #Memory split betwen the workers
    walltime = "00:10:00",
    interface= "nmn0",
    # processes = 2,
    # death_timeout=2,
    # job_extra=['--preload', 'worker_setup.py']
    # worker_command = custom_worker_command
    job_script_prologue=job_script_prologue
)

cluster.scale(3)

# from dask.distributed import Client
# client = Client(cluster)

# num_workers = len(client.scheduler_info()["workers"])
# print(num_workers)

#['lo', 'nmn0', 'hsn1', 'hsn0', 'can0', 'bond0', 'net1', 'net2', 'net3']
# cluster.scale(jobs=3)  # ask for 10 jobs


/project/project_462000451/enchanted_container_lumi3/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38467 instead
  warnings.warn(


In [3]:
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p small
#SBATCH -A project_462000451
#SBATCH -n 1
#SBATCH --cpus-per-task=10
#SBATCH --mem=954M
#SBATCH -t 00:10:00
export PATH="/project/project_462000451/enchanted_container_lumi3/bin:$PATH"
cd /project/project_462000451/enchanted-surrogates/
export PYTHONPATH=$PYTHONPATH:/project/project_462000451/enchanted-surrogates/src
/project/project_462000451/enchanted_container_lumi3/bin/python -m distributed.cli.dask_worker tcp://10.252.1.81:36769 --name dummy-name --nthreads 10 --memory-limit 0.93GiB --nanny --death-timeout 60 --interface nmn0



In [4]:
from dask.distributed import Client
client = Client(cluster)


checking file system connection


In [9]:
num_workers = len(client.scheduler_info()["workers"])
print(num_workers)

3


In [6]:
print(cluster)

SLURMCluster(d57b428c, 'tcp://10.252.1.81:36769', workers=0, threads=0, memory=0 B)


In [7]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.252.1.81:38467/status,
Dashboard: http://10.252.1.81:38467/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.252.1.81:36769,Workers: 0
Dashboard: http://10.252.1.81:38467/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [8]:
# create dask futures
from dask.distributed import print, Client, as_completed, wait
import subprocess
import os

def ls():
    print('checking file system connection')
    command = "srun --output=/project/project_462000451/DEEPlasma/std_out_echo.txt --error=/project/project_462000451/DEEPlasma/err_out_echo.txt echo 'HelloWorld!'"
    out = subprocess.run(command, capture_output=True, shell=True)
    return out
    # subprocess.run('ls /project/project_462000451/DEEPlasma')
    # print('ls', os.system('ls /project/project_462000451/DEEPlasma'))
 
new_future = client.submit(ls)
res = new_future.result()
print(res)

# futures = []
# for i, sa in enumerate(samples):
#     print(sa)
#     run_id = f'gene_dask_test_{i}'
#     # runner.code_run(sa, run_id)
#     new_future = client.submit(
#         runner.code_run, sa, run_id
#     )
#     futures.append(new_future)

CompletedProcess(args="srun --output=/project/project_462000451/DEEPlasma/std_out_echo.txt --error=/project/project_462000451/DEEPlasma/err_out_echo.txt echo 'HelloWorld!'", returncode=0, stdout=b'', stderr=b'')


In [10]:
client.shutdown()


In [21]:
def square(x):
        return x ** 2

def neg(x):
        return -x

A = client.map(square, range(10))
B = client.map(neg, A)
total = client.submit(sum, B)
total.result()

-285

In [ ]:
from dask.distributed import Client, WorkerPlugin

class MyPlugin(WorkerPlugin):
    def setup(self, worker):
        # This code runs when the worker starts up
        print(f"Worker {worker.address} is starting up")
        # You can run any command here, for example:
        # os.system('your-command-here')

# Connect to your Dask cluster
client = Client('scheduler-address:port')

# Register the plugin with the client
client.register_worker_plugin(MyPlugin())


In [1]:

from dask_jobqueue import SLURMCluster

job_script_prologue = [
    'export PATH="/project/project_462000451/enchanted_container_lumi3/bin:$PATH"',
    'cd /project/project_462000451/enchanted-surrogates/',
    'export PYTHONPATH=$PYTHONPATH:/project/project_462000451/enchanted-surrogates/src' # NB: to use the enchanted-surrogate library
]

# set these values to what you need your scientific code to run
cluster = SLURMCluster(
    queue='small',
    account="project_462000451",
    cores=1,
    memory="1GB",
    walltime = "00:10:00",
    interface= "nmn0",
    job_script_prologue=job_script_prologue,
    processes=1, # Keep this as 1
    job_extra_directives=['-o %x.%j.out','-e %x.%j.err']
    # Do not set these, leave as default
    # n_workers = 2,
    # job_cpu =10,
)
n_jobs = 4
cluster.scale(n_jobs)

/project/project_462000451/enchanted_container_lumi3/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36995 instead
  warnings.warn(


In [10]:
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p small
#SBATCH -A project_462000451
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=954M
#SBATCH -t 00:10:00
#SBATCH -o %x.%j.out
#SBATCH -e %x.%j.err
export PATH="/project/project_462000451/enchanted_container_lumi3/bin:$PATH"
cd /project/project_462000451/enchanted-surrogates/
export PYTHONPATH=$PYTHONPATH:/project/project_462000451/enchanted-surrogates/src
/project/project_462000451/enchanted_container_lumi3/bin/python -m distributed.cli.dask_worker tcp://10.120.21.15:38935 --name dummy-name --nthreads 1 --memory-limit 0.93GiB --nanny --death-timeout 60 --interface nmn0



In [2]:
from dask.distributed import Client
client = Client(cluster)

In [3]:
client

<Client: 'tcp://10.252.1.61:44843' processes=4 threads=4, memory=3.72 GiB>

In [21]:
print(client)

<Client: 'tcp://10.252.1.81:44777' processes=8 threads=800, memory=7.44 GiB>


In [33]:
from dask.distributed import Client

# Connect to the Dask cluster
# client = Client('tcp://scheduler-address:port')

# Print client information
print(client)

# Get detailed information about workers
workers_info = client.scheduler_info()['workers']
for worker, info in workers_info.items():
    print(f"Worker: {worker}")
    print(f"  Memory: {info['memory_limit'] / 1e9:.2f} GB")
    print(f"  Cores: {info['nthreads']}")
    print(f"  Host: {info['host']}")
    print(f"  Local Directory: {info['local_directory']}")
    print()


<Client: 'tcp://10.252.1.61:38679' processes=4 threads=4, memory=3.72 GiB>
Worker: tcp://10.120.20.29:35305
  Memory: 1.00 GB
  Cores: 1
  Host: 10.120.20.29
  Local Directory: /tmp/dask-scratch-space/worker-c72agrf3

Worker: tcp://10.120.20.29:40063
  Memory: 1.00 GB
  Cores: 1
  Host: 10.120.20.29
  Local Directory: /tmp/dask-scratch-space/worker-yond3owg

Worker: tcp://10.120.20.29:41283
  Memory: 1.00 GB
  Cores: 1
  Host: 10.120.20.29
  Local Directory: /tmp/dask-scratch-space/worker-ts6se_67

Worker: tcp://10.120.20.29:41835
  Memory: 1.00 GB
  Cores: 1
  Host: 10.120.20.29
  Local Directory: /tmp/dask-scratch-space/worker-fnd8pxmv

